# Deep Learning Convenience Functions

> This notebook contains convenience functions to aid in modeling. 

In [ ]:
#| default_exp dlfn

In [ ]:
#| export
import os, re
import math

from tqdm import tqdm

import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from EnvDL.core import read_json
from EnvDL.dna import np_3d_to_hilbert

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

def calc_cs(x # numeric array
           ): 
    "Calculate nan mean and nan std of an array. Returned as list"
    return [np.nanmean(x, axis = 0), np.nanstd(x, axis = 0)]

In [ ]:
#| export

def apply_cs(xs, 
             cs_dict_entry # list of length 2 containing mean and s
            ): return ((xs - cs_dict_entry[0]) / cs_dict_entry[1])

In [ ]:
#| export

def reverse_cs(xs, cs_dict_entry): return (cs_dict_entry[1] * xs) + cs_dict_entry[0]

## Train/Validate/Test Split info

Stored as json

In [ ]:
#| export

def read_split_info(
    load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/',
    json_prefix = '2023:9:5:12:8:26'):
    ""
    jsons = [e for e in os.listdir(load_from) if re.match('^'+json_prefix+'.+\.json$', e)]
    vals = [e for e in jsons if re.match('.+val\d+\.json$', e)]
    vals.sort()
    out = {}
    out['test'] = [read_json(json_path = load_from+json_prefix+'-test.json')]
    out['test_file'] = [json_prefix+'-test.json']
    out['validate'] = [read_json(json_path = load_from+val) for val in vals]
    out['validate_files'] = [val for val in vals]
    return(out)


In [ ]:
#| export

def find_idxs_split_dict(
    obs_df, # assumes presence of Year, Female, Male
    split_dict # from read_split_info() output. Should be a test of validate dict.
):
    temp = obs_df
    test_mask = ((temp.Year.isin(split_dict['test_years'])) & 
                 ((temp.Female.isin(split_dict['test_parents'])) |
                  (temp.Male.isin(split_dict['test_parents']))))
    temp['Split'] = ''
    temp.loc[test_mask, 'Split'] = 'Test'

    train_mask = (~(temp.Year.isin(split_dict['test_years'])) & 
                 (~((temp.Female.isin(split_dict['test_parents'])) |
                  (temp.Male.isin(split_dict['test_parents'])))))
    temp.loc[train_mask, 'Split'] = 'Train'

    temp_test  = (temp.Split == 'Test') # should be the same as with the mask above
    temp_train = (temp.Split == 'Train') # should be the same as with the mask above

    # Confirm that there's no overlap in parents or years
    temp_test_parents  = set(temp.loc[temp_test, 'Female']+temp.loc[temp_test, 'Male'])
    temp_train_parents = set(temp.loc[temp_train, 'Female']+temp.loc[temp_train, 'Male'])

    temp_test_years  = set(temp.loc[temp_test, 'Year'])
    temp_train_years = set(temp.loc[temp_train, 'Year'])

    assert [] == [e for e in temp_test_parents if e in temp_train_parents]
    assert [] == [e for e in temp_train_parents if e in temp_test_parents]
    assert [] == [e for e in temp_test_years if e in temp_train_years]
    assert [] == [e for e in temp_train_years if e in temp_test_years]

    return({
        'test_idx': temp.loc[test_mask, ].index, 
        'train_idx': temp.loc[train_mask, ].index} )

## Initialization

In [ ]:
#| export
def LSUV_(model, data, apply_only_to=['Conv', 'Linear', 'Bilinear'],
          std_tol=0.1, max_iters=10, do_ortho_init=True, logging_FN=print):
    r"""
    Refer to https://github.com/glassroom/torch_lsuv_init
    Applies layer sequential unit variance (LSUV), as described in
    `All you need is a good init` - Mishkin, D. et al (2015):
    https://arxiv.org/abs/1511.06422

    Args:
        model: `torch.nn.Module` object on which to apply LSUV.
        data: sample input data drawn from training dataset.
        apply_only_to: list of strings indicating target children
            modules. For example, ['Conv'] results in LSUV applied
            to children of type containing the substring 'Conv'.
        std_tol: positive number < 1.0, below which differences between
            actual and unit standard deviation are acceptable.
        max_iters: number of times to try scaling standard deviation
            of each children module's output activations.
        do_ortho_init: boolean indicating whether to apply orthogonal
            init to parameters of dim >= 2 (zero init if dim < 2).
        logging_FN: function for outputting progress information.

    Example:
        >>> model = nn.Sequential(nn.Linear(8, 2), nn.Softmax(dim=1))                                                                                                                                                                                                                                            
        >>> data = torch.randn(100, 8)
        >>> LSUV_(model, data)
    """

    matched_modules = [m for m in model.modules() if any(substr in str(type(m)) for substr in apply_only_to)]

    if do_ortho_init:
        logging_FN(f"Applying orthogonal init (zero init if dim < 2) to params in {len(matched_modules)} module(s).")
        for m in matched_modules:
            for p in m.parameters():                
                torch.nn.init.orthogonal_(p) if (p.dim() >= 2) else torch.nn.init.zeros_(p)

    logging_FN(f"Applying LSUV to {len(matched_modules)} module(s) (up to {max_iters} iters per module):")

    def _compute_and_store_LSUV_stats(m, inp, out):
        m._LSUV_stats = { 'mean': out.detach().mean(), 'std': out.detach().std() }

    was_training = model.training
    model.train()  # sets all modules to training behavior
    with torch.no_grad():
        for i, m in enumerate(matched_modules):
            with m.register_forward_hook(_compute_and_store_LSUV_stats):
                for t in range(max_iters):
                    _ = model(data)  # run data through model to get stats
                    mean, std = m._LSUV_stats['mean'], m._LSUV_stats['std']
                    if abs(std - 1.0) < std_tol:
                        break
                    m.weight.data /= (std + 1e-6)
            logging_FN(f"Module {i:2} after {(t+1):2} itr(s) | Mean:{mean:7.3f} | Std:{std:6.3f} | {type(m)}")
            delattr(m, '_LSUV_stats')

    if not was_training: model.eval()

## Network Components

In [ ]:
#| export
def Linear_block(in_size, out_size, drop_pr):
    block = nn.Sequential(
        nn.Linear(in_size, out_size),
        nn.ReLU(),
        nn.Dropout(drop_pr)
    )
    return(block) 

In [ ]:
#| export
def Conv1D_x2_Max_block(in_channels, out_channels, kernel_size, stride, maxpool_size):
    block = nn.Sequential(
        nn.Conv1d(
            in_channels= in_channels, # second channel
            out_channels= out_channels,
            kernel_size= kernel_size,
            stride= stride
        ), 
        nn.Conv1d(
            in_channels= out_channels, 
            out_channels= out_channels,
            kernel_size= kernel_size,
            stride= stride
        ), 
        nn.BatchNorm1d(out_channels),
        nn.MaxPool1d((maxpool_size), stride=stride)
    )
    return(block)

### Resnet Blocks 

#### Resnet For 4d (b, c, h, w) (2d conv)
4d is shown first because this code was adapted from the pytorch implementation of ResNet

In [ ]:
#| export
def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1):
    # Using https://pytorch.org/vision/main/_modules/torchvision/models/resnet.html as a starting point
    """3x3 convolution with padding"""

    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )

In [ ]:
#| export
def conv1x1(in_planes: int, out_planes: int, stride: int = 1):

    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [ ]:
#| export

class BasicBlock2d(nn.Module):
    expansion = 1
    def __init__(self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: nn.Module = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: nn.Module = None,
        expansion: int = 1
    ) -> None:
        super().__init__()
        # Set up defaults if none was passed in
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width !=  64: raise ValueError('Only groups = 1 and base_width = 64 supported')
        if dilation > 1: raise NotImplementedError('Dilation not supported')

        # self.expansion = expansion # in ResNet v1, this is 1, in ResNet v1.5 it is set to 4

        # self.conv1 = conv3x3(inplanes, planes*self.expansion, stride)
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        # self.conv2 = conv3x3(planes, planes*self.expansion)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [ ]:
#| export

class BottleneckBlock2d(nn.Module):
    # This is the block used in ResNet v1.5. It is supposed to be more effective.
    # Main changes are that 
    # - expansion is not set to 1 
    # - now there is a third convolution that happens in the slow path
    #
    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: nn.Module = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: nn.Module = None,
        expansion: int = 4
    ) -> None:        
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
            
        self.expansion = expansion

        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [ ]:
#| export

class ResNet2d(nn.Module):
    def __init__(
        self,
        block, #: Type[Union[BasicBlock, Bottleneck]],
        layers, #: List[int],
        num_outputs: int = 1,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation = None, #: Optional[List[bool]] = None,
        norm_layer = None, #: Optional[Callable[..., nn.Module]] = None,
        input_channels = 4
    ) -> None:
        super().__init__()
        # _log_api_usage_once(self)
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(input_channels, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False) # Note that this is 4 not 3
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_outputs)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
        self,
        block, #: Type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: torch.Tensor) -> torch.Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self._forward_impl(x)

#### Resnet For 3d (b, c, l) (1d conv)

In [ ]:
#| export

def conv0x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1):
    """3x3 convolution with padding"""
    return nn.Conv1d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


In [ ]:
#| export

def conv0x1(in_planes: int, out_planes: int, stride: int = 1):
    """1x1 convolution"""
    return nn.Conv1d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [ ]:
#| export

class BasicBlock1d(nn.Module):
    expansion = 1
    def __init__(self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: nn.Module = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: nn.Module = None,
        expansion: int = 1
    ) -> None:
        super().__init__()
        # Set up defaults if none was passed in
        if norm_layer is None:
            norm_layer = nn.BatchNorm1d
        if groups != 1 or base_width !=  64: raise ValueError('Only groups = 1 and base_width = 64 supported')
        if dilation > 1: raise NotImplementedError('Dilation not supported')

        # self.expansion = expansion # in ResNet v1, this is 1, in ResNet v1.5 it is set to 4

        # self.conv1 = conv3x3(inplanes, planes*self.expansion, stride)
        self.conv1 = conv0x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        # self.conv2 = conv3x3(planes, planes*self.expansion)
        self.conv2 = conv0x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out
  

In [ ]:
#| export

  
class BottleneckBlock1d(nn.Module):
    # This is the block used in ResNet v1.5. It is supposed to be more effective.
    # Main changes are that 
    # - expansion is not set to 1 
    # - now there is a third convolution that happens in the slow path
    #
    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: nn.Module = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: nn.Module = None,
        expansion: int = 4
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm1d
            
        self.expansion = expansion

        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv0x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv0x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv0x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out
    


In [ ]:
#| export
class ResNet1d(nn.Module):
    def __init__(
        self,
        block, #: Type[Union[BasicBlock, Bottleneck]],
        layers, #: List[int],
        num_outputs: int = 1,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation = None, #: Optional[List[bool]] = None,
        norm_layer = None, #: Optional[Callable[..., nn.Module]] = None,
        input_channels = 4
    ) -> None:
        super().__init__()
        # _log_api_usage_once(self)
        if norm_layer is None:
            norm_layer = nn.BatchNorm1d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv1d(input_channels, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False) # Note that this is 4 not 3
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool1d((1))
        self.fc = nn.Linear(512 * block.expansion, num_outputs)

        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm1d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
        self,
        block, #: Type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv0x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: torch.Tensor) -> torch.Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self._forward_impl(x)




## Training (general)

In [ ]:
#| export
def train_loop(dataloader, model, loss_fn, optimizer, silent = False):
    size = len(dataloader.dataset)
    for batch, (xs_i, y_i) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(xs_i)
        loss = loss_fn(pred, y_i) # <----------------------------------------

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(y_i) # <----------------
            if not silent:
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [ ]:
#| export

def train_error(dataloader, model, loss_fn, silent = False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss = 0

    with torch.no_grad():
        for xs_i, y_i in dataloader:
            pred = model(xs_i)
            train_loss += loss_fn(pred, y_i).item() # <----------------------
            
    train_loss /= num_batches
    return(train_loss) 

In [ ]:
#| export

def test_loop(dataloader, model, loss_fn, silent = False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for xs_i, y_i in dataloader:
            pred = model(xs_i)
            test_loss += loss_fn(pred, y_i).item() # <-----------------------

    test_loss /= num_batches
    if not silent:
        print(f"Test Error: Avg loss: {test_loss:>8f}")
    return(test_loss) 

In [ ]:
#| export

def yhat_loop(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    y_true = np.array([])
    y_pred = np.array([])
    
    with torch.no_grad():
        for xs_i, y_i in dataloader:
            yhat_i = model(xs_i)
            y_pred = np.append(y_pred, np.array(yhat_i.cpu()))
            y_true = np.append(y_true, np.array(y_i.cpu()))
    
    out = np.concatenate([y_true[:, None], y_pred[:, None]], axis = 1) 
    out = pd.DataFrame(out, columns = ['y_true', 'y_pred'])
    return(out)

In [ ]:
#| export

def train_nn(
    cache_path,
    training_dataloader,
    testing_dataloader,
    model,
    learning_rate = 1e-3,
    batch_size = 64,
    epochs = 500,
    model_prefix = 'model'
):
    # Initialize the loss function
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    loss_df = pd.DataFrame([i for i in range(epochs)], columns = ['Epoch'])
    loss_df['TrainMSE'] = np.nan
    loss_df['TestMSE']  = np.nan

    for t in tqdm(range(epochs)):        
        # print(f"Epoch {t+1}\n-------------------------------")
        train_loop(training_dataloader, model, loss_fn, optimizer, silent = True)

        loss_df.loc[loss_df.index == t, 'TrainMSE'
                   ] = train_error(training_dataloader, model, loss_fn, silent = True)
        
        loss_df.loc[loss_df.index == t, 'TestMSE'
                   ] = test_loop(testing_dataloader, model, loss_fn, silent = True)
        
        if (t+1)%5 == 0: # Cache in case training is interupted. 
            # print(loss_df.loc[loss_df.index == t, ['TrainMSE', 'TestMSE']])
            torch.save(model.state_dict(), 
                       cache_path+'/'+model_prefix+'_'+str(t)+'_'+str(epochs)+'.pt') # convention is to use .pt or .pth
        
    return([model, loss_df])

In [ ]:
#| export

def estimate_iterations(sec_per_it = 161):
    hours = [1, 2, 4, 8, 12, 24]
    res = pd.DataFrame(zip(hours, 
    [math.floor(
        ((i)*(60*60))/sec_per_it
    ) for i in hours]), columns = ['Hours', 'Iterations'])
    return(res)

In [ ]:
#| export
class ACGTDataset(Dataset): # for any G containing matix with many (phno) to one (geno)
    def __init__(self, 
                 y, 
                 G, # not on gpu
                 idx_original,
                 idx_lookup,
                 transform = None, target_transform = None,
                 use_gpu_num = 0,
                 device = 'cuda',
                 **kwargs 
                ):

        self.device = device
        self.y = y 
        self.G = G
        self.idx_original = idx_original
        self.idx_lookup = idx_lookup
        self.transform = transform
        self.target_transform = target_transform    
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        y_idx =self.y[idx]
            
        #                 |array containing correct index in deduplicated g 
        #                 |               index in phno    
        uniq_g_idx = self.idx_lookup[self.idx_original[idx], 1]
        g_idx = self.G[uniq_g_idx, :, :]
        
        # send all to gpu        
        if (self.device != 'cpu'):
            if y_idx.device.type == 'cpu':
                y_idx = y_idx.to(self.device) 
                
            if g_idx.device.type == 'cpu':
                g_idx = g_idx.to(self.device) 
        
        
        if self.transform:
            g_idx = self.transform(g_idx)
            
        if self.target_transform:
            y_idx = self.transform(y_idx)
        return g_idx, y_idx

## For Multiple Inputs (but not requiring multiple inputs)

In [ ]:
#| export
class BigDataset(Dataset):
    def __init__(
        self,
        lookup_obs,
#         lookup_geno,
#         lookup_env,
#         y,
#         G, 
#         G_type
#         S,
#         P,
#         W,
#         W_type,
        transform = None, 
        target_transform = None,
        **kwargs 
        ):
        """
        This class produces a set with one or more input tensors. For flexibility the only _required_ input is `lookup_obs`, a tensor with the index of observations. 
        Everything else is provided as a kwarg. Output is a list of tensors1 ordered [y, G, S, W], any of these not initalized will be missing but not empty (e.g. [y, S, W] not [y, None, S, W]).       
        Used inputs are:
        lookup_obs: index for y, used by __getitem__ for obs_idx
        lookup_geno: index for G, row obs_idx, column 1 is geno_idx (geno information is deduplicated, hence the need for a lookup)
        lookup_env: index for S & W, , row obs_idx, column 1 is env_idx (env information is deduplicated, hence the need for a lookup)
        y: yield
        G: Genomic information 
        G_type: how the infomation should be returned, 'raw', 'hilbert', or 'list' (i.e. of tensors for snps in each gene)
        S: Soil information
        P: Planting/Harvest date contained in column 0, 1 respectively 
        W: Weather data
        W_type: how the infomation should be returned, 'raw' or 'hilbert'

        1 G may also be returned as a list of tensors
        """
        # Lookup info (so that deduplication works)
        self.lookup_obs = lookup_obs
        # if 'lookup_obs'  in kwargs: self.lookup_obs  = kwargs['lookup_obs'];
        if 'lookup_geno' in kwargs: self.lookup_geno = kwargs['lookup_geno'];
        if 'lookup_env'  in kwargs: self.lookup_env  = kwargs['lookup_env'];
        # Data
        if 'y' in kwargs: self.y = kwargs['y'];
        if 'G' in kwargs: self.G = kwargs['G'];
        if 'S' in kwargs: self.S = kwargs['S'];
        if 'P' in kwargs: self.P = kwargs['P']; # PlantHarvest so that planting can be added into W
        if 'W' in kwargs: self.W = kwargs['W'];
        # Data prep state information
        if 'G_type' in kwargs: self.G_type = kwargs['G_type']; # raw, hilbert, list
        if 'W_type' in kwargs: self.W_type = kwargs['W_type']; # raw, hilbert
        # Data to be returned
        self.out_names = [e for e in ['y', 'G', 'S', 'W'] if e in kwargs]
        # Transformations
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.lookup_obs)
    

    # These used to be in __getitem__ but separating them like this allows for them to be overwritten more easily
    def get_y(self, idx):
        y_idx = self.y[idx]
        if self.transform:
            y_idx = self.transform(y_idx)
        return(y_idx)
        
    def get_G(self, idx):
        geno_idx = self.lookup_geno[idx, 1]
        if self.G_type in ['raw', 'hilbert']:
            G_idx = self.G[geno_idx]
        if 'list' == self.G_type:
            G_idx = [e[geno_idx] for e in self.G]
        if self.transform:
            G_idx = self.transform(G_idx)
        return(G_idx)

    def get_S(self, idx):
        env_idx = self.lookup_env[idx, 1]
        S_idx = self.S[env_idx]
        if self.transform:
            S_idx = self.transform(S_idx)
        return(S_idx)

    def get_W(self, idx):
        W_device = torch.Tensor(self.W).get_device()


        env_idx = self.lookup_env[idx, 1]
        # get growing information
        WPlant = np.zeros(365)
        # WPlant[self.P[obs_idx, 0]:self.P[obs_idx, 1]] = 1
        WPlant[self.P[idx, 0]:self.P[idx, 1]] = 1
        if self.W_type == 'raw':
            WPlant = torch.from_numpy(WPlant).to(torch.float)
            # if needed send to gpu
            if W_device != -1: WPlant = WPlant.to(W_device)            
            W_idx = torch.concatenate([self.W[env_idx], WPlant[None, :]], axis = 0)
        if self.W_type == 'hilbert':
            # convert growing info to hilbert curve
            WPlant_hilb = np_3d_to_hilbert(WPlant[None, :, None], silent = True)
            WPlant_hilb = WPlant_hilb.squeeze(axis = 3)
            WPlant_hilb[np.isnan(WPlant_hilb)] = 0
            WPlant_hilb = torch.from_numpy(WPlant_hilb).to(torch.float)
            # if needed send to gpu
            if W_device != -1: WPlant_hilb = WPlant_hilb.to(W_device)
            W_idx = torch.concatenate([self.W[env_idx], WPlant_hilb], axis = 0)
        if self.transform:
            W_idx = self.transform(W_idx)
        return(W_idx)
    
    def __getitem__(self, idx):
        out = []
        # obs_idx = self.lookup_obs[idx]
        if 'y' in self.out_names: out += [self.get_y(idx)]
        if 'G' in self.out_names: out += [self.get_G(idx)]
        if 'S' in self.out_names: out += [self.get_S(idx)]
        if 'W' in self.out_names: out += [self.get_W(idx)]
        return out


In [ ]:

# demo = BigDataset(
#     lookup_obs = [i for i in range(len(YMat))], # Training/testing. To use a deduplicated dataset, 
#                                                 # Pass the training index through the desired lookup
#                                                 # list(set(obs_env_lookup[:, 2])) -> phno idxs for each env (1 for each)
#     y = YMat,
#     lookup_geno = obs_geno_lookup,
#     G = ACGT,
#     G_type = 'raw',
#     lookup_env = obs_env_lookup,
#     S = SMat,
#     P = PlantHarvest,
#     W = WMat,
#     W_type = 'raw',
#           )   

In [ ]:
#| export

# Standard data prep

# Wrapper function to hide the steps of loading data
import numpy as np
from EnvDL.core import get_cached_result
from EnvDL.dlfn import read_split_info, find_idxs_split_dict
import pandas as pd
import torch

class g2fc_datawrapper():   
    def __init__(self):
        self.data_dict = {}
        self.cs_dict = {}
        print('Loading and storing default `phno`.')
        self.load(name='phno', store = True)
    

    def set_split(self, load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/', json_prefix = '2023:9:5:12:8:26'):
        if 'phno' not in self.data_dict.keys():
            print('`phno` must be stored!\nManually initialize with .load()')
        else:
            split_info = read_split_info(load_from = load_from, json_prefix = json_prefix)

            temp = self.data_dict['phno'].copy()
            temp[['Female', 'Male']] = temp['Hybrid'].str.split('/', expand = True)

            self.test_dict = find_idxs_split_dict(
                obs_df = temp, 
                split_dict = split_info['test'][0]
            )

            temp = temp.loc[self.test_dict['train_idx'], ] # restrict before re-aplying

            self.val_dict = find_idxs_split_dict(
                obs_df = temp, 
                split_dict = split_info['validate'][0]
            )

    def generic_load(self, load_from, file_name):        
        if   file_name.split('.')[-1] == 'pkl': res = get_cached_result(load_from+file_name)
        elif file_name.split('.')[-1] == 'npy': res = np.load(load_from+file_name)
        elif file_name.split('.')[-1] == 'csv': res = pd.read_csv(load_from+file_name)
        else: print(f'Unrecognized file encoding: {file_name.split(".")[-1]} \nReturning None'); res = None
        return res

    def load(self, name='ACGT', store = False, **kwargs):
        # defaults for quick access
        defaults_dict = {
            ## Genomic Data
            'ACGT':         ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'ACGT.npy'],
            'ACGT_hilb':    ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'ACGT_hilb.npy'],            
            'KEGG_entries': ['../nbs_artifacts/01.05_g2fc_demo_model/', 'filtered_kegg_gene_entries.pkl'],
            'KEGG_slices':  ['../nbs_artifacts/01.05_g2fc_demo_model/', 'ACGT_gene_slice_list.pkl'],

            ## Soil and Management 
            'mgmtMatNames': ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'mgmtMatNames.npy'],
            'mgmtMat':      ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'mgmtMat.npy'],
            'SMatNames':    ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'SMatNames.npy'],
            'SMat':         ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'SMat.npy'],

            ## Weather
            'PlantHarvestNames': ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'PlantHarvestNames.npy'],
            'PlantHarvest':      ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'PlantHarvest.npy'],
            'WMat':              ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'WMat.npy'],
            'WMatNames':         ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'WMatNames.npy'],
            'WMat_hilb':         ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'WMat_hilb.npy'],

            # Response and lookup
            'phno':            ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'phno_geno.csv'],
            'obs_geno_lookup': ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'obs_geno_lookup.npy'], # Phno_Idx  Geno_Idx  Is_Phno_Idx
            'obs_env_lookup':  ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'obs_env_lookup.npy'],  # Phno_Idx  Env_Idx   Is_Phno_Idx
            'YMat':            ['../nbs_artifacts/01.03_g2fc_prep_matrices/', 'YMat.npy']
        }

        if name in defaults_dict.keys():
            load_from, file_name = defaults_dict[name]
        else: 
            print(f'`name` not recognized. Use `load_from` and `file_name` for greater control.\,Allowed `names` are:\n{list(defaults_dict.keys())}')
        
        # overwrite defaults if desired
        if 'load_from' in kwargs.keys(): load_from = kwargs['load_from']
        if 'file_name' in kwargs.keys(): file_name = kwargs['file_name']

        res = self.generic_load(load_from=load_from, file_name= file_name)

        if store:
            self.data_dict[name] = res
        else:
            return res

    def load_all(self, name_list = [], store = False):
        if store:
            for e in name_list:
                self.load(name = e, store=store)
        else:
            res_list = []
            for e in name_list:
                res_list += [self.load(name = e, store=store)]
            return res_list

    def store_cs(self, name, cs_list):
        self.cs_dict[name] = cs_list

    def calc_cs(self, name, version = 'np', **kwargs):

        res = self.data_dict[name]
        if 'filter' in kwargs.keys():
            which_dict, which_split = kwargs['filter'].split(':')

            if which_split == 'train':  key = 'train_idx'
            elif which_split == 'test': key = 'test_idx'
            else: print('only `train` and `test` indexes are allowed.')

            if which_dict == 'val':    mask = self.val_dict[key]
            elif which_dict == 'test': mask = self.test_dict[key]
            else: print('only `val` and `test` sets are allowed.')

            if 'filter_lookup' in kwargs.keys():
                # This block exists because some data is deduplicated. In the dataloader I use lookup tables to find the right values.
                # That gets messy because the enviroment, genome, and yield all get different ones
                # I could hardcode names to filters but that would make this code pretty inflexible (which I would like to avoid.)
                # using the manual overwrite metod .store_cs() it's possible to get the desired behavior like this:
                # X.store_cs('WMat', calc_cs(X.get('WMat')[np.array(list(set(X.get('obs_env_lookup', ops_string='filter:val:train')[:, 1]))),: ,:]))
                # That is a lot messier looking than I would like. It's hard to see what's happening. 
                # To get around this I'm adding a 'filter_lookup' kwarg that does the same job as the lookup tables in the data loader.
                lookup = self.data_dict[kwargs['filter_lookup']]
                lookup = lookup[mask, 1]
                # deduplicate; for cs we don't need the order of the obs.
                mask = np.array(list(set(lookup)))
            res = res[mask]

        else:
            print('''
Scaling based on ALL data. To avoid this pass in a split to be used. 
If a lookup table should be used to select observations (e.g. obs_env_lookup ) its name should be passed in. 
E.g. filter = \'val:train\',  filter_lookup = \'obs_env_lookup\'
                  ''')

        if version == 'np':
            self.cs_dict[name] = [np.asarray(np.mean(res, axis = 0)), np.asarray(np.std(res, axis = 0))]
        elif version == 'torch':
            self.cs_dict[name] = [torch.Tensor.mean(res, axis = 0), torch.Tensor.std(res, axis = 0)]

    def calc_cs_all(self, name_list, version = 'np', **kwargs):
        for name in name_list:
            self.calc_cs(name=name, version = version, **kwargs)

    def apply_cs(self, name, **kwargs):
        if name not in self.cs_dict.keys():
            self.calc_cs(name, kwargs)

        vals = self.cs_dict[name]
        res = self.data_dict[name]
        
        if type(res) == type(vals[0]):
            pass
        elif type(res) == torch.Tensor:
            # convert to pytorch
            vals = [torch.from_numpy(e) for e in vals]
        elif type(res) == np.ndarray:
            # convert to numpy
            vals = [torch.Tensor.numpy(e) for e in vals]
            
        center, scale = vals
        res = (res - center) / scale
        return res

    def get_cs(self, name):
        if name not in self.cs_dict.keys():
            res = None
        else:
            res = self.cs_dict[name]
        return res

    def reverse_cs(self, name, x):
        vals = self.cs_dict[name]

        if type(x) == type(vals[0]):
            pass
        elif type(x) == torch.Tensor:
            # convert to pytorch
            vals = [torch.from_numpy(e) for e in vals]
        elif type(res) == np.ndarray:
            # convert to numpy
            vals = [torch.Tensor.numpy(e) for e in vals]

        center, scale = vals
        res = (res * scale) + center
        return res

    
    def get(self, name, ops_string = ''):
        # is split info being requested? (for lookup_obs most likely). Otherwise main data is being requested.
        if name not in ['val:train', 'test:train', 'val:test', 'test:test']:
            if name not in self.data_dict.keys():
                self.load(name, store=True)
            res = self.data_dict[name]
        else: 
            which_dict, which_split = name.split(':')

            if which_split == 'train':  key = 'train_idx'
            elif which_split == 'test': key = 'test_idx'
            else: print('only `train` and `test` indexes are allowed.')

            if which_dict == 'val':    res = self.val_dict[key]
            elif which_dict == 'test': res = self.test_dict[key]
            else: print('only `val` and `test` sets are allowed.')
            
        # apply opperations
        ops_string = [e for e in ops_string.split(' ') if e != '']
        for ops in ops_string:
            if ops[0:6] == 'filter':
                _, which_dict, which_split = ops.split(':')

                if which_split == 'train':  key = 'train_idx'
                elif which_split == 'test': key = 'test_idx'
                else: print('only `train` and `test` indexes are allowed.')

                if which_dict == 'val':    res_idx = self.val_dict[key]
                elif which_dict == 'test': res_idx = self.test_dict[key]
                else: print('only `val` and `test` sets are allowed.')

                res = res[res_idx]

            if ops == 'cs':
                res = self.apply_cs(name)
            
            if ops == 'asarray':
                res = np.asarray(res)
            if ops == 'from_numpy':
                res = torch.from_numpy(res)
            if ops == 'float':
                res = res.to(torch.float)
            if ops[0:4] == 'cuda':
                # send to device by number. e.g. cuda:0 -> X.to(0)
                res = res.to(int(ops.split(':')[-1]))

        return res


# some example usage 
# X = g2fc_datawrapper()
# X.set_split()
# X.load_all(name_list = ['obs_env_lookup', 'YMat', 'PlantHarvest', 'WMat',], store=True) 
# X.calc_cs('YMat', version = 'np', filter = 'val:train'); X.cs_dict['YMat']
# X.calc_cs_all(['YMat'], version = 'np', filter = 'val:train'); X.cs_dict['YMat']
# X.calc_cs('YMat', version = 'np'); X.cs_dict['YMat']

# some demonstration of when to use kwargs for scaling 
# how do I manually do scaling for enviromental things?
# X.store_cs('WMat', calc_cs(X.get('WMat')[np.array(list(set(X.get('obs_env_lookup', ops_string='filter:val:train')[:, 1]))),: ,:]))
# [e[0:3, 0] for e in X.cs_dict['WMat']]
# X.calc_cs('WMat', filter = 'val:train', filter_lookup= 'obs_env_lookup')
# [e[0:3, 0] for e in X.cs_dict['WMat']]
# X.calc_cs('WMat')
# [e[0:3, 0] for e in X.cs_dict['WMat']]

# X.get('WMat', ops_string='asarray')[0:3, 0:3, 0]
# X.get('WMat', ops_string='cs asarray')[0:3, 0:3, 0]


## Functions for Visible Neural Nets (y first)

In [ ]:
#| export

def train_loop_yx(dataloader, model, loss_fn, optimizer, silent = False):
    size = len(dataloader.dataset)
    for batch, (y_i, xs_i) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(xs_i)
        
        # ensure both are on cuda
        if pred.device.type == 'cpu':
            pred = pred.to('cuda')
        if y_i.device.type == 'cpu':
            y_i = y_i.to('cuda')
        
        loss = loss_fn(pred, y_i)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(y_i) 
            if not silent:
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
#| export

def train_error_yx(dataloader, model, loss_fn, silent = False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss = 0

    with torch.no_grad():
        for y_i, xs_i in dataloader:
            pred = model(xs_i)
            
            # ensure both are on cuda
            if pred.device.type == 'cpu':
                pred = pred.to('cuda')
            if y_i.device.type == 'cpu':
                y_i = y_i.to('cuda')
            
            train_loss += loss_fn(pred, y_i).item()
            
    train_loss /= num_batches
    return(train_loss)

In [ ]:
#| export

def test_loop_yx(dataloader, model, loss_fn, silent = False):   
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for y_i, xs_i in dataloader:
            pred = model(xs_i)
            
            # ensure both are on cuda
            if pred.device.type == 'cpu':
                pred = pred.to('cuda')
            if y_i.device.type == 'cpu':
                y_i = y_i.to('cuda')
                
            test_loss += loss_fn(pred, y_i).item() 

    test_loss /= num_batches
    if not silent:
        print(f"Test Error: Avg loss: {test_loss:>8f}")
    return(test_loss)

In [ ]:
#| export

def train_nn_yx(
    cache_path,
    training_dataloader,
    testing_dataloader,
    model,
    batch_size = 64,
    epochs = 500,
    model_prefix = 'model',
    save_model = False,
    **kwargs # can include 'silent' for train loop or 'save_on' for saving frequency
):   
    if 'optimizer' not in kwargs:
        optimizer = torch.optim.SGD(model.parameters(), lr=kwargs['learning_rate'])
    else:
        optimizer = kwargs['optimizer']
        
    if 'save_on' in kwargs:
        save_on = kwargs['save_on']
    else:
        save_on = 5       
    
    # Initialize the loss function
    loss_fn = nn.MSELoss()     

    loss_df = pd.DataFrame([i for i in range(epochs)], columns = ['Epoch'])
    loss_df['TrainMSE'] = np.nan
    loss_df['TestMSE']  = np.nan

    for t in tqdm(range(epochs)):        
        if 'silent' in kwargs:
            train_loop_yx(training_dataloader, model, loss_fn, optimizer, silent = kwargs['silent'])
        else:
            train_loop_yx(training_dataloader, model, loss_fn, optimizer, silent = True)

        loss_df.loc[loss_df.index == t, 'TrainMSE'
                   ] = train_error_yx(training_dataloader, model, loss_fn, silent = True)
        
        loss_df.loc[loss_df.index == t, 'TestMSE'
                   ] = test_loop_yx(testing_dataloader, model, loss_fn, silent = True)
        
        if (t+1)%save_on == 0: # Cache in case training is interupted. 
            if save_model:
                torch.save(model.state_dict(), 
                           cache_path+'/'+model_prefix+'_'+str(t)+'_'+str(epochs)+'.pt') # convention is to use .pt or .pth
        
    return([model, loss_df])

In [ ]:
#| export

def yhat_loop_yx(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    y_true = np.array([])
    y_pred = np.array([])
    
    with torch.no_grad():
        for y_i, xs_i in dataloader:
            yhat_i = model(xs_i)
            y_pred = np.append(y_pred, np.array(yhat_i.cpu()))
            y_true = np.append(y_true, np.array(y_i.cpu()))
    
    out = np.concatenate([y_true[:, None], y_pred[:, None]], axis = 1) 
    out = pd.DataFrame(out, columns = ['y_true', 'y_pred'])
    return(out)

## Functions from multi-trait output tests

In [ ]:
# #| export

# def train_loop3(dataloader, model, loss_fn, optimizer, silent = False):
#     "This is a version of train_loop which concatenates three ys."
#     import torch
#     from torch.utils.data import Dataset
#     from torch.utils.data import DataLoader
#     size = len(dataloader.dataset)
#     for batch, (xs_i, y1_i, y2_i, y3_i) in enumerate(dataloader):
#         # Compute prediction and loss
#         pred = model(xs_i)
#         loss = loss_fn(pred, torch.concat([y1_i, y2_i, y3_i], axis = 1)) # <----------------------------------------

#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         if batch % 100 == 0:
#             loss, current = loss.item(), batch * len(y1_i) # <----------------
#             if not silent:
#                 print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

                

In [ ]:
# #| export

# def train_error3(dataloader, model, loss_fn, silent = False):
#     import torch
#     from torch.utils.data import Dataset
#     from torch.utils.data import DataLoader
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     train_loss = 0

#     with torch.no_grad():
#         for xs_i, y1_i, y2_i, y3_i in dataloader:
#             pred = model(xs_i)
#             train_loss += loss_fn(pred, torch.concat([y1_i, y2_i, y3_i], axis = 1)).item() # <----------------------
            
#     train_loss /= num_batches
#     return(train_loss) 

In [ ]:
# #| export

# def test_loop3(dataloader, model, loss_fn, silent = False):
#     import torch
#     from torch.utils.data import Dataset
#     from torch.utils.data import DataLoader

#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     test_loss = 0

#     with torch.no_grad():
#         for xs_i, y1_i, y2_i, y3_i in dataloader:
#             pred = model(xs_i)
#             test_loss += loss_fn(pred, torch.concat([y1_i, y2_i, y3_i], axis = 1)).item() # <-----------------------

#     test_loss /= num_batches
#     if not silent:
#         print(f"Test Error: Avg loss: {test_loss:>8f}")
#     return(test_loss) 

In [ ]:
# #| export

# def yhat_loop3(dataloader, model):
#     "Version of yhat_loop that returns 3 ys"
#     import numpy as np
#     import pandas as pd
#     import torch
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
    
#     first_loop = True
    
#     with torch.no_grad():
#         for xs_i, y1_i, y2_i, y3_i in dataloader:
#             yhat_i = model(xs_i)
#             y_i = torch.concat([y1_i, y2_i, y3_i], axis = 1) # <-----------------------

#             if first_loop:
#                 y_pred = np.array(yhat_i.cpu())
#                 y_true = np.array(y_i.cpu())
#                 first_loop = False
#             else:            
#                 y_pred = np.concatenate([y_pred, np.array(yhat_i.cpu())])
#                 y_true = np.concatenate([y_true, np.array(y_i.cpu())])
                
#     out = np.concatenate([y_true[:, :], y_pred[:, :]], axis = 1) 
#     out = pd.DataFrame(out, columns = ['y1_true', 'y2_true', 'y3_true', 'y1_pred', 'y2_pred', 'y3_pred'])
#     return(out)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()